# Encrypting AES BIE1

### Bitcoin elliptic curve conventions

In [1]:
import binascii
import hashlib
import ecdsa
import codecs

# secp256k1, http://www.oid-info.com/get/1.3.132.0.10
p_ec = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
r_ec = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
b_ec = 0x0000000000000000000000000000000000000000000000000000000000000007
a_ec = 0x0000000000000000000000000000000000000000000000000000000000000000
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
curve_secp256k1 = ecdsa.ellipticcurve.CurveFp(p_ec, a_ec, b_ec)
generator_secp256k1 = ecdsa.ellipticcurve.Point(curve_secp256k1, Gx, Gy, r_ec)
oid_secp256k1 = (1, 3, 132, 0, 10)
SECP256k1 = ecdsa.curves.Curve("SECP256k1", curve_secp256k1, generator_secp256k1, oid_secp256k1)

### Establishing a shared secret

Shared point 

In [135]:
SA = 68391376603652558019677085321245066310756998499824362348774548882158845001985
PA = SA * generator_secp256k1

SB = 99700726144837597339837499243820914618717618365214734329132998527433625976967
PB = SB * generator_secp256k1

In [140]:
print(PA.x())
print(PA.y())

12299240128971589684193058412293321166201311702354403913508722600796858365069
111633549770467382038798363592268494989008286422683433990409403137059100396092


In [142]:
PA = ecdsa.ellipticcurve.Point(ecdsa.SECP256k1.curve, 12299240128971589684193058412293321166201311702354403913508722600796858365069, 111633549770467382038798363592268494989008286422683433990409403137059100396092)

In [143]:
PA.x()

12299240128971589684193058412293321166201311702354403913508722600796858365069

In [3]:
# Shared secret

SAB = SB * PB
SAB.x()

24547207253151602529617225665887624910292130993218504679930071992194063220891

In [4]:
# Check shared secrets are equal if calculated by either Alice or Bob

(SA * PB) == (SB * PA)

True

Shared data

In [5]:
def get_point_pubkey(point_input):
    if (point_input.y() % 2) == 1:
        result = '03' + '%064x' % point_input.x()
    else:
        result = '02' + '%064x' % point_input.x()
    return binascii.hexlify(codecs.decode(result, 'hex'))

In [6]:
S = get_point_pubkey(SAB)
S

b'0236453a194181e14a3333f9e284afefffc05eec8680e25c6bf5432aa53d65409b'

### Symmetric keys

In [10]:
IV = hashlib.sha512(S).digest()[0:16]
KAES = hashlib.sha512(S).digest()[16:32]
KHMAC = hashlib.sha512(S).digest()[32:48]

### Encrypt message

## Working solution

In [17]:
aes = AES.new(KAES, AES.MODE_CBC, IV)
message = b'The answer is no'    # I think this is ascii encoded bits
print(message)
print(binascii.hexlify(message))
ciphertext = aes.encrypt(message)
print(binascii.hexlify(ciphertext))
print(binascii.hexlify(aes.decrypt(IV + ciphertext)[16:]))
print(aes.decrypt(IV + ciphertext)[16:])

b'The answer is no'
b'54686520616e73776572206973206e6f'
b'acf4eb9dba1920fcd598bb9669f20ae9'
b'54686520616e73776572206973206e6f'
b'The answer is no'


In [18]:
# Check that we could have encrypted the message directly

aes = AES.new(KAES, AES.MODE_CBC, IV)
message = b'54686520616e73776572206973206e6f'    # I think this is ascii encoded bits
print(message)
ciphertext = aes.encrypt(message)
print(binascii.hexlify(ciphertext))
print(aes.decrypt(IV + ciphertext)[16:])

b'54686520616e73776572206973206e6f'
b'6d622ce583dee1c4618a026ced39dea07fd95e756af53a12f41869eda78e6dba'
b'54686520616e73776572206973206e6f'


# Electrum test

In [160]:
import base64
import base58

### Working solution

In [177]:
entire_package = 'QklFMQOzklE/S7Y7q25EbIRID6+l84voNQ5huAqcGgmKFYk39EJyEjlPQZ6tcYtBKRikhPe6phPsScTU5NvUUQnZc5HoHg/JRZ0qijLGvueEgMkEor03ruO4cxk0WsxClF13450='


entire_package = binascii.hexlify(base64.b64decode(entire_package))

print(entire_package)

b'4249453103b392513f4bb63bab6e446c84480fafa5f38be8350e61b80a9c1a098a158937f4427212394f419ead718b412918a484f7baa613ec49c4d4e4dbd45109d97391e81e0fc9459d2a8a32c6bee78480c904a2bd37aee3b87319345acc42945d77e39d'


In [190]:
# Split off protocol flag, ephemeral pub key, ciphertext and HMAC

BIE1 = entire_package[:8]
K = entire_package[8:74]             # 8 + 66 = 74  
ciphertext = entire_package[74:98]   # 74 + 24 = 98
HMAC = entire_package[98:122]         # 98 + 24 = 122

print(BIE1, 'in ascii is', binascii.unhexlify(BIE1))
print('K is', K)
print('ciphertext is', ciphertext)
print('HMAC is', HMAC)

b'42494531' in ascii is b'BIE1'
K is b'03b392513f4bb63bab6e446c84480fafa5f38be8350e61b80a9c1a098a158937f4'
ciphertext is b'427212394f419ead718b4129'
HMAC is b'18a484f7baa613ec49c4d4e4'


 ### Import  keys

Bob's private key'

In [179]:
priv = binascii.hexlify(base64.b64decode('L4cBnoBky32Drn56STohahrd7hutuKuDD5MJFbSTS53U6sLoR2zC'))
print(priv)                        

pub = binascii.hexlify(codecs.decode('03f1fda2e261389cca99c13d85e7c27549d1f59891519aa1c533eb794aa187f58c', 'hex'))
print(pub)

b'2f87019e8064cb7d83ae7e7a493a216a1addee1badb8ab830f930915b4934b9dd4eac2e8476cc2'
b'03f1fda2e261389cca99c13d85e7c27549d1f59891519aa1c533eb794aa187f58c'


In [180]:
# Private key is in wallet import format
# https://en.bitcoin.it/wiki/Wallet_import_format

priv = binascii.hexlify(base58.b58decode('L4cBnoBky32Drn56STohahrd7hutuKuDD5MJFbSTS53U6sLoR2zC'))
print(priv) 
priv = priv[:-8]  #  remove last 4 bytes which is a checksum
print(priv)
priv = priv[2:]   # remove first byte which is mainnet/testnet flag
print(priv)
priv = priv[:-2]  #  remove last bytes which is a compressed public key flag
print(priv)
print(len(priv))

b'80dc6c9c7a71c025a50238e4ca4ac33443cb88eb368a1ac64db8149d854cf2408701af4c9599'
b'80dc6c9c7a71c025a50238e4ca4ac33443cb88eb368a1ac64db8149d854cf2408701'
b'dc6c9c7a71c025a50238e4ca4ac33443cb88eb368a1ac64db8149d854cf2408701'
b'dc6c9c7a71c025a50238e4ca4ac33443cb88eb368a1ac64db8149d854cf24087'
64


In [181]:
#SB = int.from_bytes(priv, "little")  
SB = int(priv, 16)
print(SB)

PB = SB * generator_secp256k1

CompPB = get_point_pubkey(PB)

print(CompPB)

99700726144837597339837499243820914618717618365214734329132998527433625976967
b'03f1fda2e261389cca99c13d85e7c27549d1f59891519aa1c533eb794aa187f58c'


Ephemeral public key created by alice

In [191]:
K

b'03b392513f4bb63bab6e446c84480fafa5f38be8350e61b80a9c1a098a158937f4'

In [192]:
# Taken from https://stackoverflow.com/questions/43629265/deriving-an-ecdsa-uncompressed-public-key-from-a-compressed-one

def pow_mod(x, y, z):
    "Calculate (x ** y) % z efficiently."
    number = 1
    while y:
        if y & 1:
            number = number * x % z
        y >>= 1
        x = x * x % z
    return number

# prime p = 2^256 - 2^32 - 2^9 - 2^8 - 2^7 - 2^6 - 2^4 - 1
p = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f

# bitcoin's compressed public key of private key 99700726144837597339837499243820914618717618365214734329132998527433625976967
compressed_key = '03b392513f4bb63bab6e446c84480fafa5f38be8350e61b80a9c1a098a158937f4'

y_parity = int(compressed_key[:2]) - 2
x = int(compressed_key[2:], 16)

a = (pow_mod(x, 3, p) + 7) % p
y = pow_mod(a, (p+1)//4, p)

if y % 2 != y_parity:
    y = -y % p

uncompressed_key = '04{:x}{:x}'.format(x, y)
print(uncompressed_key) 

04b392513f4bb63bab6e446c84480fafa5f38be8350e61b80a9c1a098a158937f4383335171d29cc6387a5c18d5ccb57e859ff38f3fa1bb8e00475f093cadddfeb


In [193]:
PA = ecdsa.ellipticcurve.Point(ecdsa.SECP256k1.curve, int(uncompressed_key[2:66], 16), int(uncompressed_key[66:130], 16))

Shared data

In [194]:
print(SB)

99700726144837597339837499243820914618717618365214734329132998527433625976967


In [195]:
SAB = SB * PA
S = get_point_pubkey(SAB)

In [196]:
IV = hashlib.sha512(S).digest()[0:16]
KAES = hashlib.sha512(S).digest()[16:32]
KHMAC = hashlib.sha512(S).digest()[32:48]

Encrypt

In [197]:
from Crypto.Cipher import AES

In [198]:
aes = AES.new(KAES, AES.MODE_CBC, IV)
message = b'The answer is no'    # I think this is ascii encoded bits
print(message)
print(binascii.hexlify(message))
ciphertext = aes.encrypt(message)
print(binascii.hexlify(ciphertext))

b'The answer is no'
b'54686520616e73776572206973206e6f'
b'6a98d4e90fa40c48cbe709853c9b0628'
